In [2]:
# This program aims to find out all combinations of data segments that have no 'position collisions' 
# A 'position collision' means if we could find the same digit at the same position when comparing two binary numbers


def getPosition(String):           # Obtain the positions where '1's occupy in a binary number (in the string form)
    position_list = []             # e.g. 1000000010000000 = [0, 8]
    for i in range (0, 16):        # The binary string has been segmented with a block length of 16
        if String[i] == '1':       # The returned result is consideredas a 'data position block'
            position_list.append(i)
    return position_list


def collision(List1, List2):       # Detect repeated data positions across data position blocks
    count = 0                      # e.g. Block [0, 7] and [0, 8] have data position '0' in common, therefore those two blocks have one collisions
    for List1_element in List1:
        for List2_element in List2:
            if List1_element == List2_element:             
                count += 1 
                break
        if (count != 0):
            break
    return count


def repeatScreening(List):        # Exclude repeated blocks in a combination list 
    newList = []
    for sublist in List:
        if sublist not in newList:
            newList.append(sublist)
    return newList
 
    
def combinationScreening_3(List):    # Find out all combinations containing 3 orthogonal blocks
    combination_3 = []
    for List1 in List:
        for List2 in List:
            for List3 in List:
                if collision(List1, List2) == 0 and collision(List1, List3) == 0 and collision(List2, List3) == 0:
                    combination_3_unit = [List1, List2, List3]
                    combination_3.append(combination_3_unit)
    return combination_3
 
    
def combinationScreening_2(List):    # Find out all combinations containing 2 orthogonal blocks
    combination_2 = []
    for List1 in List:
        for List2 in List:
            if collision(List1, List2) == 0:
                combination_2_unit = [List1, List2]
                combination_2.append(combination_2_unit)
    return combination_2


def eliminateNone(List):         # Eliminate position blocks which has been converted to 'None'
    for List1 in List:
        for element in List1:
            if element == None:
                List1.remove(element)
    return List
    
    
def duplicateScreening_3(List):      # Eliminate duplicate data position blocks in the list 
    screened = []
    count = 0
    while(len(List) > 0):
        screened.append(List[0])
  
        List.remove(List[0])
        for i in range (0, len(screened[count])):
            for j in range (0, len(List)):
                for j1 in range (0, len(List[j])):
                    if screened[count][i] == List[j][j1]:
                        List[j][j1] = None                  # Convert duplicate position block to a 'None'
        
        List = [List1 for List1 in List if List1 != [None, None, None]]     # Eliminate combinations with all blocks converted to 'None', leaving combinations with partial blocks converted
  
        List = eliminateNone(List)
      
        List = eliminateNone(List)          # Need to repeat the 'None' element elimination step because in a combination there are 3 position blocks and there may be two 'None's after screening 
      
        for i in range (0, len(List)):                
            if len(List[i]) == 1:                       # To obtain the minimal possible numbers of combinations for a dataset, we need to keep combinations with just 1 block as few as possible 
                for j in range (i+1, len(List)):        # Therefore in the duplicate elimination step, if we have two combinations, one has just 1 block while the other has 2 blocks, and the single block happens to be a duplicate, we need to remove this single block instead of eliminating the block in the 2-block combination
                    for j1 in range (0, len(List[j])):  # e.g. Combination A = [[0, 1, 2, 3]]
                        if List[i][0] == List[j][j1]:   #      Combination B = [[0, 1, 2, 3], [4, 5, 6, 7]]
                            List[i][0] = None           #      After screening, the result should be A = [None], B = B instead of A = A, B = [None, [4, 5, 6 ,7]] 
    
        List = eliminateNone(List)
   
        List = [List1 for List1 in List if List1 != []]    # Remove all empty lists
        count += 1

    screened = [List1 for List1 in screened if len(List1) == 3]     # Keep all lists with 3 position blocks

    return screened   


def duplicateScreening_2(List):     # For duplicate screening with 2 position blocks
    screened = []
    count = 0
    while(len(List) > 0):
        screened.append(List[0])
   
        List.remove(List[0])
        for i in range (0, len(screened[count])):
            for j in range (0, len(List)):
                for j1 in range (0, len(List[j])):
                    if screened[count][i] == List[j][j1]:
                        List[j][j1] = None
        
        List = [List1 for List1 in List if List1 != [None, None]]

        List = eliminateNone(List)
 
        for i in range (0, len(List)):
            if len(List[i]) == 1:
                for j in range (i+1, len(List)):
                    for j1 in range (0, len(List[j])):
                        if List[i][0] == List[j][j1]:
                            List[i][0] = None
       
        List = eliminateNone(List)
       
        List = [List1 for List1 in List if List1 != []]
        count += 1

    screened = [List1 for List1 in screened if len(List1) == 2]
   
    return screened   


def eliminate(List1, List2):    # Eliminate position blocks which has been grouped into Combinatinn_3 or Combination_2
    for List21 in List2:
        for List22 in List21:
            List1 = [List11 for List11 in List1 if List11 != List22]
    return List1
        

# Whole hex data after addition of '20' to each segment
%store -r my_hexdata

# Conversion to binary form
scale = 16
num_of_bits = 8
my_hexdata_binary = bin(int(my_hexdata, scale))[2:].zfill(num_of_bits)
my_hexdata_binary = '0'*(4-len(my_hexdata_binary)%4) + my_hexdata_binary 

combination_candidates = []
for i in range (0, int(len(my_hexdata_binary)/16)):
    ready_to_getPosition = my_hexdata_binary[i*16:(i+1)*16]
    combination_candidates.append(getPosition(ready_to_getPosition))

combination_candidates = repeatScreening(combination_candidates)

data_combination_3 = combinationScreening_3(combination_candidates)
data_combination_3 = duplicateScreening_3(data_combination_3)

combination_candidates = eliminate(combination_candidates, data_combination_3)
data_combination_2 = combinationScreening_2(combination_candidates)
data_combination_2 = duplicateScreening_2(data_combination_2)
combination_candidates = eliminate(combination_candidates, data_combination_2)

data_combination_1 = [[None]*1 for _ in range(len(combination_candidates))]
for i in range (0, len(combination_candidates)):
    data_combination_1[i][0] = combination_candidates[i]

Groups = data_combination_1 + data_combination_2 + data_combination_3

print(len(data_combination_1))
print(len(data_combination_2))
print(len(data_combination_3))

print(len(Groups))

%store Groups


640
76
1
717
Stored 'Groups' (list)
